In [1]:
import pandas as pd
import numpy as np
from huggingface_hub import login, logout
from datasets import load_dataset
from util.util_data import tokenize
from transformers import AutoTokenizer, AutoConfig, pipeline
from transformers import Trainer, TrainingArguments

In [3]:
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("quant-chef/BioPharmaCatalysts", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
dataset = dataset.map(tokenize(tokenizer=tokenizer), batched=True)

TypeError: tokenize() missing 1 required positional argument: 'doc'

In [2]:
train = pd.read_csv('./out/train.csv')

In [52]:
train.loc[train.label == 1].iloc[3]

ticker                                                    SNY
disease                                      COVID-19 vaccine
stage                                                 Phase 3
date                                      2021-12-15 00:00:00
catalyst    Phase 3 data reported that the vaccine is well...
label                                                       1
Name: 4, dtype: object